## Berekenen van de NDVI

De Normalized Difference Vegetation Index (NDVI) wordt gebruikt om een inschatting te geven van de gezondheid van de vegetatie op een bepaalde locatie. Voor de NDVI berekening heb je de reflectie in het rood en in het infrarood nodig. Deze spectrale gegevens worden ingewonnen met remote sensingvaak vanuit de ruimte met observatiesatellieten. Voor de Sentinel2 satelliet zijn dit de banden 04 (rood) en 08 (infrarood). 

De NDVI wordt berekend met de volgende formule uitgaande van de Sentinel2 banden voor rood en infrarood:

NDVI = (B08 + B04) / (B08 - B04)

De NDVI heeft een waarde tussen de +1 en -1. Hoe dichter de NDVI bij +1 ligt, hoe gezonder de vegetatie.

De onderstaande functie `calculate_ndvi` berekent de NDVI. Omdat we hierbij gebruikmaken van `GDAL` moet ook deze module geïmporteerd worden.

In [ ]:
# Importeer noodzakelijke modules
from osgeo import gdal
from osgeo import gdalnumeric
from osgeo import gdalconst
from matplotlib import pyplot
import os
import numpy

# Function to calculate NDVI rasterimage
def calculate_ndvi (rasterfile_red_name, rasterfile_nir_name) :
    
    try:
    
        # Construct name of NDVI rasterfile
        rasterfile_ndvi_name = rasterfile_red_name.replace('B04', 'NDVI')
        
        # Open Red band en schrijf data naar array
        rasterfile_red = gdal.Open(rasterfile_red_name, gdalconst.GA_ReadOnly )
        rasterband_red = rasterfile_red.GetRasterBand(1)
        array_red      = gdalnumeric.BandReadAsArray(rasterband_red)
        
        # Open NIR band en schrijf data naar array
        rasterfile_nir = gdal.Open(rasterfile_nir_name, gdalconst.GA_ReadOnly )
        rasterband_nir = rasterfile_nir.GetRasterBand(1)
        array_nir      = gdalnumeric.BandReadAsArray(rasterband_nir)
        
        # Bereken NDVI: Let op: Maak er eerst een array met floats van omdat resultaat een float is
        array_ndvi = (array_nir.astype(float) - array_red.astype(float))/(array_nir.astype(float) + array_red.astype(float))
        
        # Haal driver op voor output GeoTIFF raster bestand 
        driver = gdal.GetDriverByName("GTiff")
    
        # Haal aantal rijen en kolommen op voor output raster bestand
        nr_of_columns = rasterfile_red.RasterXSize
        nr_of_rows    = rasterfile_red.RasterYSize
        
        # Maak leeg raster bestand met datatype float32
        rasterfile_ndvi = driver.Create(rasterfile_ndvi_name, nr_of_columns, nr_of_rows, 1, gdal.GDT_Float32)

        # Kopieer geotrans en coordinaatsysteem
        gdalnumeric.CopyDatasetInfo(rasterfile_red, rasterfile_ndvi)

        # Haal band op en schrijf array naar band
        rasterband_ndvi = rasterfile_ndvi.GetRasterBand(1)
        gdalnumeric.BandWriteArray(rasterband_ndvi, array_ndvi)

        # Sluiten van bestanden
        array_red = None
        rasterfile_red  = None  
        array_nir = None
        rasterfile_nir  = None
        array_ndvi = None
        rasterfile_ndvi = None

        # Print name
        print(rasterfile_ndvi_name + ' generated')

        # Return name of NDVI file
        return rasterfile_ndvi_name

    except Exception as e: 
        
        try:
            rasterfile_red = None
            rasterfile_nir = None
            rasterfile_ndvi = None
        except:
            None

        print('Error function transform_raster_image')
        print(e)

Nu kan de functie `calculate_ndvi` gebruikt worden om de NDVI te berekenen. De berekende NDVI kaart wordt geplot en er wordt een histogram van de NDVI waardes gemaakt.

In [ ]:
# Navigeer naar de "data" folder met de satellietbeelden
os.chdir('data_files')

# Bereken de NDVI
file_name_B04 = "T31UFT_20190824T105031_B04_10m.tif"
file_name_B08 = "T31UFT_20190824T105031_B08_10m.tif"
file_name_NDVI = calculate_ndvi(file_name_B04, file_name_B08)
print('Naam van bestand met berekende NDVI is ' + str(file_name_NDVI))

# Open rasterbestand en kopieer data uit band naar array
raster_bestand = gdal.Open(file_name_NDVI, gdalconst.GA_ReadOnly)
rasterband     = raster_bestand.GetRasterBand(1)
array_band     = gdalnumeric.BandReadAsArray(rasterband)

# Laad array in pyplot en toon plot met colormap pink
pyplot.figure(figsize=(20,10))
pyplot.imshow(array_band, cmap='Reds')
pyplot.show()

# Show histogram
aantal_klassen = 50
pyplot.hist(numpy.ravel(array_band),aantal_klassen)
pyplot.title('Histogram van rasterband')
pyplot.xlabel('DN'); 
pyplot.ylabel('Frequentie')
pyplot.show()

# Sluit bestand
raster_bestand = None